In [1]:
import sys
sys.path.insert(0, '../../wildlife-tools')
sys.path.insert(0, '..')

import matplotlib.pyplot as plt
import pandas as pd
import torchvision.transforms as T
import timm

from wildlife_datasets import datasets
from wildlife_tools.data import WildlifeDataset
from wildlife_tools.features import DeepFeatures
from wildlife_tools.similarity import CosineSimilarity
from wildlife_tools.inference import KnnClassifier


c:\Users\sadda\Downloads\Github\wildlife-datasets\venv_wt\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
qwe = 0
for d_name in datasets.names_all:
    try:
        #print(d_name.__name__, int(d_name.metadata['reported_n_individuals']))
        #d_name.metadata
        qwe += d_name.metadata['reported_n_individuals']
    except:
        print(d_name)

        pass
qwe

<class 'wildlife_datasets.datasets.datasets.Giraffes'>
<class 'wildlife_datasets.datasets.datasets.HappyWhale'>
<class 'wildlife_datasets.datasets.datasets.HumpbackWhaleID'>
<class 'wildlife_datasets.datasets.datasets.NOAARightWhale'>
<class 'wildlife_datasets.datasets.datasets.SarahZelvy'>
<class 'wildlife_datasets.datasets.datasets.StripeSpotter'>
<class 'wildlife_datasets.datasets.datasets.ZindiTurtleRecall'>


6612.0

In [14]:
datasets.HappyWhale.metadata

{'url': 'https://www.kaggle.com/competitions/happy-whale-and-dolphin',
 'cite': 'cheeseman2017happywhale',
 'animals': {'dolphin', 'whale'},
 'real_animals': True,
 'year': 2022,
 'wild': True,
 'clear_photos': True,
 'pose': 'multiple',
 'unique_pattern': True,
 'from_video': False,
 'cropped': False,
 'span': 'very long',
 'size': 61912.0}

In [5]:
d_name.metadata

{'url': 'ftp://pbil.univ-lyon1.fr/pub/datasets/miele2021',
 'cite': 'miele2021revisiting',
 'animals': {'giraffe'},
 'real_animals': True,
 'year': 2021,
 'wild': True,
 'clear_photos': True,
 'pose': 'double',
 'unique_pattern': True,
 'from_video': True,
 'cropped': True,
 'span': 'unknown',
 'size': 1719.0}

In [ ]:
root = 'data/FriesianCattle2015'
datasets.FriesianCattle2015.get_data(root)

In [ ]:
d = datasets.FriesianCattle2015(root)
dataset = WildlifeDataset(d.df, d.root)

In [ ]:
dataset.plot_grid(n_cols=5);

In [ ]:
dataset = WildlifeDataset(d.df, d.root, img_load='crop_black')
dataset.plot_grid(n_cols=5);

In [ ]:
plt.plot(dataset.metadata['identity'].value_counts().values)

In [ ]:
# Show how various datasets were created - FriesianCattle2015 / Drosophila

In [ ]:
model_name = 'hf-hub:BVRA/MegaDescriptor-T-224'
model = timm.create_model(model_name, num_classes=0, pretrained=True)
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
extractor = DeepFeatures(model)

In [ ]:
df1 = dataset.metadata[dataset.metadata['identity'] == 1]
df2 = dataset.metadata[dataset.metadata['identity'] == 2]
df_database = pd.concat((df1.iloc[2:], df2.iloc[2:]))
df_query = pd.concat((df1.iloc[:2], df2.iloc[:2]))


In [ ]:
d_database = WildlifeDataset(df_database, d.root, transform=transform, img_load='crop_black')
d_query = WildlifeDataset(df_query, d.root, transform=transform, img_load='crop_black')

In [ ]:
d_query.plot_grid(n_cols=5);

In [ ]:
database = extractor(d_database)
query = extractor(d_query)

In [ ]:
query.shape

In [ ]:
similarity_function = CosineSimilarity()
similarity = similarity_function(query, database)['cosine']

In [ ]:
similarity[:2,:]

In [ ]:
similarity[2:,:]

In [ ]:
classifier = KnnClassifier(k=1, database_labels=d_database.labels_string)
predictions = classifier(similarity)
predictions

In [ ]:
idx = (-similarity).argsort(axis=-1)[:, :5]

In [ ]:
idx_true = df_query.index
idx_pred = df_database.index.to_numpy()[idx]

dataset.plot_predictions(idx_true, idx_pred);